In [22]:
%run EconIndicators.ipynb

In [2]:
# Getting Training Economic Indicators from World Bank (From EconIndicators.ipynb)
raw_data = getAllIndicatorsTraining(20, full_country_basket)
norm_data = relativeStrength(raw_data)

In [ ]:
norm_data

In [28]:
a = getAllIndicators(full_country_basket)

Retrying API call for: GDC in 10 seconds.


In [29]:
a

,GDP Year,GDP,GDG Year,GDG,GDC Year,GDC,CPI Year,CPI,UEM Year,UEM
AUS,2019,1.396567e+12,2019,2.160956,2019,55060.326101,2019,1.610768,2020,6.460000
BRA,2019,1.839758e+12,2019,1.136586,2019,8717.186278,2019,3.732976,2019,11.930000
CAN,2019,1.736426e+12,2019,1.656041,2019,46194.725226,2019,1.949269,2020,9.460000
CHE,2019,7.030824e+11,2019,0.928832,2019,81993.727126,2019,0.362916,2019,4.390000
CHN,2019,1.434290e+13,2019,6.110118,2019,10261.679245,2019,2.899236,2019,5.150000
EUU,2019,1.562645e+13,2019,1.554265,2019,34918.498375,2019,1.630523,2019,6.700782
GBR,2019,2.829108e+12,2019,1.462658,2019,42330.117537,2019,1.738105,2019,3.740000
IND,2019,2.868929e+12,2019,4.180728,2019,2099.599048,2019,7.659695,2019,5.270000
JPN,2019,5.081770e+12,2019,0.654152,2019,40246.880128,2019,0.476974,2019,2.400000
KOR,2019,1.646739e+12,2019,2.039348,2019,31846.218232,2019,0.382946,2020,3.930000
